# Interactuando con SpreadSheets de google

Documentación del paquete [pygsheets](https://pygsheets.readthedocs.io/en/staging/)

## Descargo los ids de los cursos

In [31]:
import pygsheets

gc = pygsheets.authorize()

In [80]:
sh = gc.open('CURSOS-CONTENIDOS-REQUISITOS-OBJETIVOS')
hoja_web = sh.worksheet_by_title('Para web')

In [33]:
cursos_ids = hoja_web.range("G2:G122", returnas="range")

In [4]:
cursos_ids = [curso[0].value for curso in cursos_ids.cells]

In [5]:
cursos_ids

['79',
 '78',
 '479',
 '77',
 '38',
 '92',
 '85',
 '163',
 '57',
 '931',
 '919',
 '914',
 '61',
 '84',
 '1047',
 '488',
 '329',
 '39',
 '109',
 '1037',
 '168',
 '41',
 '42',
 '43',
 '955',
 '112',
 '4',
 '489',
 '476',
 '44',
 '6',
 '5',
 '90',
 '3',
 '45',
 '46',
 '47',
 '48',
 '49',
 '307',
 '374',
 '209',
 '167',
 '166',
 '254',
 '426',
 '165',
 '925',
 '86',
 '920',
 '40',
 '111',
 '253',
 '7',
 '99',
 '304',
 '252',
 '260',
 '52',
 '53',
 '55',
 '427',
 '923',
 '56',
 '164',
 '1049',
 '54',
 '81',
 '160',
 '173',
 '159',
 '174',
 '171',
 '169',
 '172',
 '178',
 '179',
 '422',
 '421',
 '944',
 '946',
 '947',
 '948',
 '949',
 '950',
 '928',
 '1044',
 '160',
 '173',
 '1050',
 '174',
 '169',
 '178',
 '179',
 '1053',
 '422',
 '1038',
 '1039',
 '1040',
 '1041',
 '1042',
 '1046',
 '171',
 '1042',
 '1051',
 '421',
 '1052',
 '1043',
 '483',
 '1055',
 '1048',
 '1090',
 '1094',
 '1095',
 '1096',
 '1097',
 '1098',
 '1099',
 '0',
 '87',
 '7']

## Obteniendo la imagen de cabecera de los cursos

In [1]:
import mechanicalsoup
from dotenv import load_dotenv
import os

load_dotenv()

ADMIN_USER = os.getenv("AULARAGON_ADMIN_USER")
PASSWORD = os.getenv("AULARAGON_ADMIN_PASSWORD")

In [7]:
def get_ms_admin_browser():
    browser = mechanicalsoup.StatefulBrowser()
    browser.open("https://moodle.catedu.es/login/index.php")
    browser.select_form()
    browser["username"] = ADMIN_USER
    browser["password"] = PASSWORD
    browser.submit_selected()
    return browser

def get_course_image(browser, course_id):
    browser.open(f"https://moodle.catedu.es/course/view.php?id={course_id}")
    page = browser.get_current_page()
    course_image = page.find("div", class_="course-header-image")
    image_url = course_image.get('style').split('url("')[1].split('"); -webkit')[0]
    return course_id, image_url
    
no_image = ['', '#N/A', '0']
course_images = {}
browser = get_ms_admin_browser()
for curso in cursos_ids:
    if curso not in no_image:
        curso_id, curso_image = get_course_image(browser, curso)
        course_images[curso_id] = curso_image

In [8]:
len(course_images)

109

## Añadiendo la imagen de portada a la hoja de cálculo

In [12]:
cursos_imagenes_url= hoja_web.range("G2:H122", returnas="range")

In [ ]:
for row in cursos_imagenes_url:
    try:
        if row[0].value in list(course_images.keys()):
            row[1].set_value(course_images[row[0].value])
    except:
        print("no sé qué ha pasado")

## Obteniendo etiquetas

In [18]:
import moodle_admin.moodle_api as mp
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [1]:
courses = mp.call("core_course_get_courses")
df = pd.DataFrame(courses)

In [2]:
df_categorias = pd.read_csv("listado_cursos.csv")

In [3]:
df_categorias

,id,category,shortname,fullname,visible
0,1,0,Aularagón,Aularagón,1
1,3,Top/Cursos activos/AULARAGON/Destrezas digital...,LIBREOFFICE,LibreOffice: La suite ofimática libre,1
2,4,Top/Cursos activos/AULARAGON/Otros,ING-DUOLINGO,Inglés Duolingo,1
3,5,Top/Cursos activos/AULARAGON/STEAM/Pensamiento...,Radio,La Radio en la Escuela,1
4,6,Top/Cursos activos/AULARAGON/STEAM/Pensamiento...,FLAUTA,La enseñanza-aprendizaje de la flauta escolar,1
...,...,...,...,...,...
787,1096,Top/Cursos activos/AULARAGON/Itinerarios forma...,MascDiver_VisiIdent_EduAfecSex,MASCULINIDADES DIVERSAS. VISIBILIZACIÓN DE IDE...,1
788,1097,Top/Cursos activos/AULARAGON/Itinerarios forma...,ErraViol,ERRADICACIÓN DE LA VIOLENCIA,1
789,1098,Top/Cursos activos/AULARAGON/Itinerarios forma...,Intersec_ConsIden_Cult,INTERSECCIONALIDAD. CONSTRUCCIÓN DE IDENTIDADE...,1
790,1099,Top/Cursos activos/AULARAGON/Itinerarios forma...,ConvIgualdad,CONVIVENCIA,1


In [4]:
df_categorias = df_categorias[df_categorias["category"].str.contains("/AULARAGON/")]

In [19]:
df_categorias["etiquetas"] = df_categorias["category"].str.split("/AULARAGON/").str[1].str.split("/").str.join(",")

In [35]:
df_etiquetas = df_categorias[["id","etiquetas"]]

## Añadiendo etiquetas

In [81]:
lista_etiquetas = {str(item['id']): item['etiquetas'] for item in df_etiquetas.to_dict('records')}

In [ ]:
cursos_etiquetas_google = hoja_web.range("E91:G122", returnas="range")
for row in cursos_etiquetas_google:
    try:
        row[0].set_value(lista_etiquetas[row[2].value])
    except:
        row[0].set_value("Otros")